<a href="https://colab.research.google.com/github/dylanpatel78/Stock-Recommender-Engine/blob/main/Stock_Recommender_Engine_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Dependencies

In [23]:
import requests
import pandas as pd
from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import os
from google.colab import userdata
import yfinance as yf
import requests

# Fetching the Stock Data

In [24]:
# Getting my News Api Key from Google Colab Secrets
NEWS_API_KEY = userdata.get('newsapikey')

# Fetching major stocks in all sectors or using sector-based ETFs
def fetch_all_stocks_in_sector(sector):
    sector_mapping = {
         "Technology": ["AAPL", "MSFT", "GOOGL", "META", "TSLA", "NVDA", "ORCL", "ADBE", "CRM", "INTC", "IBM", "AMD"],
         "Healthcare": ["JNJ", "PFE", "MRK", "ABT", "UNH", "CVS", "LLY", "BMY", "MDT", "GILD", "AMGN", "ANTM"],
         "Financials": ["JPM", "BAC", "WFC", "C", "GS", "MS", "AXP", "BK", "USB", "PNC", "SCHW", "TFC"],
         "Consumer Discretionary": ["AMZN", "HD", "MCD", "SBUX", "NKE", "LOW", "TGT", "TJX", "DG", "YUM", "EBAY"],
         "Energy": ["XOM", "CVX", "BP", "TTE", "COP", "SLB", "HAL", "PSX", "VLO", "OKE", "MPC"],
         "Industrials": ["BA", "MMM", "HON", "UPS", "GE", "LMT", "CAT", "DE", "GD", "UNP", "RTX", "FDX"],
         "Real Estate": ["AMT", "PLD", "CBRE", "SPG", "PSA", "O", "EQIX", "WELL", "EXR", "VTR"],
         "Materials": ["LIN", "APD", "ECL", "NEM", "SHW", "DD", "PPG", "VMC", "FCX", "IFF"],
         "Utilities": ["NEE", "DUK", "SO", "D", "AEP", "EXC", "SRE", "ES", "PEG", "XEL"],
         "Communication Services": ["GOOG", "DIS", "NFLX", "CMCSA", "T", "VZ", "TMUS", "CHTR", "EA", "OMC"],
         "Consumer Staples": ["PG", "KO", "PEP", "WMT", "COST", "PM", "MO", "CL", "KMB", "MDLZ"]
    }
    return sector_mapping.get(sector, [])

# Fetching stock data using yfinance
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period="3mo")  # Fetching 3 months of historical data
        info = stock.info

        if df.empty:
            print(f"No data available for {symbol}")
            return None, None  # Returning None for both stock data and info if no data
        return df, info
    except Exception as e:
        print(f"Error fetching data for {symbol} from Yahoo Finance: {e}")
        return None, None


# Doing the Sentiment Analysis


In [25]:
from textblob import TextBlob
import tweepy

client = tweepy.Client(bearer_token=userdata.get('twitterbearertoken'))

def analyze_sentiment(tweets):
    total_polarity = 0
    count = 0

    # Looping through the tweets and analyze sentiment
    for tweet in tweets:
        analysis = TextBlob(tweet.text)
        polarity = analysis.sentiment.polarity  # Polarity ranges from -1 (negative) to 1 (positive)
        total_polarity += polarity
        count += 1

    # Returning the average polarity
    if count > 0:
        return total_polarity / count
    else:
        return 0  # Neutralizing the sentiment if no tweets are found

def get_stock_sentiment(symbol):
    try:
        response = client.search_recent_tweets(query=symbol, max_results=100, tweet_fields=['text', 'lang'])
        tweets = response.data
        if tweets:
            sentiment = analyze_sentiment(tweets)
            return sentiment
        else:
            print(f"No tweets found for {symbol}")
            return 0  # Returning the neutral sentiment if no tweets have been found
    except Exception as e:
        print(f"Error fetching tweets for {symbol}: {e}")
        return 0  # Returning the neutral sentiment if there's an error

# Fetching stock news using the NewsAPI
def fetch_stock_news(symbol):
    url = f"https://newsapi.org/v2/everything?q={symbol}&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        print(f"Error fetching news for {symbol}: {response.status_code}")
        return None

# Analyzing the  sentiment based on news articles
def analyze_sentiment(articles):
    total_polarity = 0
    count = 0
    for article in articles:
        if article['description']:  # Ensuring the the article has a description
            analysis = TextBlob(article['description'])
            total_polarity += analysis.sentiment.polarity
            count += 1
    return total_polarity / count if count > 0 else 0 # Returning the average polarity



# Ranking the Stocks


In [27]:
# Ranking stocks based on the performance and sentiment analysis
def rank_stocks(stocks, budget):
    ranked_stocks = []
    for stock in stocks:
        df, info = fetch_stock_data(stock)
        if df is not None and info is not None:
            latest_close = df['Close'].iloc[-1]
            if latest_close <= budget:
                sentiment = analyze_sentiment(fetch_stock_news(stock))  # Getting the news sentiment
                performance = df['Close'].iloc[-1] / df['Close'].iloc[-30]
                score = (0.7 * performance) + (0.3 * sentiment)  # Using the scoring strategy
                shares = budget // latest_close  # Calculating number of shares

                # Appending the additional info: stock ticker, company name, score, price, shares
                ranked_stocks.append({
                    'ticker': stock,
                    'name': info.get('longName', 'Unknown'),
                    'score': score,
                    'price': latest_close,
                    'shares_to_buy': shares
                })
        else:
            print(f"Not enough data for {stock}. Skipping.")

    ranked_stocks.sort(key=lambda x: x['score'], reverse=True)  # Sorting by score based on the formula
    return ranked_stocks[:4]  # Returning top 4 stocks to the user

# Asking for input from the user
sector = input("Enter the sector you're interested in (e.g., Technology, Healthcare, Financials, Energy, etc.): ")
budget = float(input("Enter your budget (in $): "))

# Fetching all the stocks based on the chosen sector
stocks = fetch_all_stocks_in_sector(sector)

# Ranking the stocks based on performance and sentiment
top_stocks = rank_stocks(stocks, budget)

# Displaying the top stocks to the user
if top_stocks:
    print(f"With a budget of ${budget}, you can afford the following stocks:")
    for stock in top_stocks:
        print(f"Company: {stock['name']}, Ticker: {stock['ticker']}, Price per Share: ${stock['price']:.2f}, Shares to Buy: {int(stock['shares_to_buy'])}, Score: {stock['score']:.2f}")
else:
    print("No suitable stocks found based on your budget and sector.")


Enter the sector you're interested in (e.g., Technology, Healthcare, Financials, Energy, etc.): Energy
Enter your budget (in $): 5000
With a budget of $5000.0, you can afford the following stocks:
Company: ONEOK, Inc., Ticker: OKE, Price per Share: $93.46, Shares to Buy: 53, Score: 0.84
Company: TotalEnergies SE, Ticker: TTE, Price per Share: $68.07, Shares to Buy: 73, Score: 0.75
Company: Chevron Corporation, Ticker: CVX, Price per Share: $141.87, Shares to Buy: 35, Score: 0.73
Company: ConocoPhillips, Ticker: COP, Price per Share: $105.51, Shares to Buy: 47, Score: 0.73
